In [13]:
import pandas as pd
import os
import yaml
import numpy as np
import json
from itertools import combinations
import pretty_midi


In [2]:
# Load JSON w/ chords
json_path = './data/jsb-chorales-8th.json'
with open(json_path, "r") as f:
     chorale_data = json.loads(f.read())
print(len(chorale_data['train']),len(chorale_data['test']),len(chorale_data['valid']))

229 77 76


In [8]:
pitch_to_name = {
    0:'C',
    1:'C#/Db',
    2:'D',
    3:'D#/Eb',
    4:'E',
    5:'F',
    6:'F#/Gb',
    7:'G',
    8:'G#/Ab',
    9:'A',
    10:'A#/Bb',
    11:'B'
}
def note_name_from_number(pitch):
    octave = (pitch//12) - 1
    pitch = pitch%12
    return pitch_to_name[pitch] + str (octave)

In [20]:
SOUNDFONT_PATH = "./GeneralUser_GS_v1.471.sf2"

def midi_to_wav(midi_path,wav_path):
    cmd = "fluidsynth -F " + wav_path + ' ' + SOUNDFONT_PATH + ' ' + midi_path + ' -r 16000 -i'
    ret_status = os.system(cmd)
    if ret_status != 0:
        sys.exit(ret_status)

def chorale_to_MIDI(chorale, synth=True, mid_path='chorale.mid'):
    midi_obj = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=1)
    cur_time=0
    for i in range(len(chorale)): 
        for note in chorale[i]: 
            cur_note = pretty_midi.Note(velocity=100, pitch=note, start=cur_time, end=cur_time+0.5)
            instrument.notes.append(cur_note)
        cur_time += 0.5
    midi_obj.instruments.append(instrument)
    midi_obj.write(mid_path)
    if synth: 
        midi_to_wav(mid_path, 'chorale.wav')

chorale_to_MIDI(chorale_data['train'][1])


FluidSynth runtime version 2.1.9
Copyright (C) 2000-2021 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'chorale.wav'..


fluidsynth: panic: An error occurred while reading from stdin.


In [21]:
def print_notes(chorale):
    for chord in chorale: 
        notes = []
        for note in chord: 
            notes.append(note_name_from_number(note))
        print(notes)

print_notes(chorale_data['train'][2])

'''for chorale in chorale_data['train']:
    print(len(chorale))
    print_notes(chorale)
    break'''

['G4', 'D4', 'B3', 'G3']
['G4', 'D4', 'B3', 'G3']
['G4', 'E4', 'C4', 'C3']
['G4', 'E4', 'C4', 'C3']
['A4', 'D4', 'F#3', 'C4']
['A4', 'D4', 'F#3', 'C4']
['G4', 'D4', 'B3', 'G3']
['G4', 'D4', 'B3', 'G3']
['A4', 'E4', 'C4', 'A3']
['B4', 'E4', 'C4', 'A3']
['C5', 'F4', 'C4', 'G#3']
['C5', 'F4', 'D4', 'G#3']
['C5', 'G4', 'E4', 'G3']
['C5', 'G4', 'E4', 'G3']
['B4', 'G4', 'D4', 'G3']
['B4', 'G4', 'D4', 'G3']
['C5', 'G4', 'E4', 'C3']
['C5', 'G4', 'E4', 'C3']
['C5', 'G4', 'E4', 'C3']
['C5', 'G4', 'E4', 'C3']
['D5', 'B4', 'G4', 'G3']
['D5', 'A4', 'F4', 'G3']
['E5', 'B4', 'E4', 'G#3']
['E5', 'B4', 'E4', 'G#3']
['C5', 'A4', 'E4', 'A3']
['C5', 'A4', 'E4', 'A3']
['C5', 'A4', 'E4', 'A3']
['C5', 'A4', 'E4', 'A3']
['B4', 'G4', 'E4', 'E3']
['C5', 'G4', 'E4', 'E3']
['D5', 'G4', 'D4', 'B2']
['D5', 'G4', 'D4', 'C3']
['A4', 'F#4', 'D4', 'D3']
['A4', 'F#4', 'D4', 'D3']
['A4', 'F#4', 'D4', 'D3']
['A4', 'F#4', 'D4', 'D3']
['D5', 'F#4', 'A3', 'D3']
['D5', 'F#4', 'A3', 'D3']
['D5', 'G4', 'B3', 'G3']
['D5', 'G4', 

"for chorale in chorale_data['train']:\n    print(len(chorale))\n    print_notes(chorale)\n    break"

In [66]:
notes_in_keys = {
    #        C D E F G A B
    "C":    [0,2,4,5,7,9,11], # C!
    "G":    [0,2,4,6,7,9,11], #  F#
    "D":    [1,2,4,6,7,9,11], # +C#
    "A":    [1,2,4,6,8,9,11], # +G#
    "E":    [1,3,4,6,8,9,11], # +D#
    "B/Cb": [1,3,4,6,8,10,11], # +A#
    "Gb/F#":[1,3,5,6,8,10,11], # +E#
    "Db/C#":[0,1,3,5,6,8,10], # +B#
    "Ab":   [0,1,3,5,7,8,10], # +Db
    "Eb":   [0,2,3,5,7,9,10], # +Ab
    "Bb":   [0,2,3,5,7,9,10], # +Eb 
    "F":    [0,2,4,5,7,9,10]  # +Bb
}

accidental_list = [1,3,6,8,10]
flats_in_order = [10,3,8,1,6,11,4]
sharps_in_order = [6,1,8,3,10,5,0]
accidental_dict = {
    0:0,
    1:1,
    2:0,
    3:1,
    4:0,
    5:0,
    6:1,
    7:0,
    8:1,
    9:0,
    10:1,
    11:0
}

key_by_number_flats = {
    0:'C',
    1:'F',
    2:'Bb',
    3:'Eb',
    4:'Ab',
    5:'Db',
    6:'Gb',
}

key_by_number_sharps = {
    0:'C',
    1:'G',
    2:'D',
    3:'A',
    4:'E',
    5:'B',
    6:'F#',
}

    
def key_estimate(chorale):
    # Chorale is a list of lists of 4 pitches representing chords
    note_counts = {}
    for chord in chorale: 
        for note in chord: 
            cur_note = note%12
            if cur_note in note_counts.keys():
                note_counts[cur_note] +=1
            else: 
                note_counts[cur_note] =1
    sorted_notecounts = {k: v for k, v in sorted(note_counts.items(), key=lambda item: item[1], reverse=True)}
    top_7 = sorted(int(i) for i in list(sorted_notecounts.keys())[0:7])
    accidentals = list(set(top_7) & set(accidental_list))
    key = -1

    for potential_key in notes_in_keys.keys(): 
        if set(top_7) == set(notes_in_keys[potential_key]):
            print("FOUND ONE!", potential_key)
            return potential_key
        
    if set(accidentals) == set(flats_in_order[:len(accidentals)]):
        key = key_by_number_flats[len(accidentals)]
        return key
    elif set(accidentals) == set(sharps_in_order[:len(accidentals)]):
        key = key_by_number_sharps[len(accidentals)]
        return key
    
    print("NOTHING WORKED")
    return key

failed = 0
for i in range(len(chorale_data['train'])):
    key = key_estimate(chorale_data['train'][i])
    if key == -1:
        failed +=1
print(failed) # this method does not work for 38 chorales


FOUND ONE! Eb
FOUND ONE! A
FOUND ONE! G
FOUND ONE! C
FOUND ONE! C
NOTHING WORKED
NOTHING WORKED
FOUND ONE! F
FOUND ONE! E
NOTHING WORKED
FOUND ONE! D
FOUND ONE! Eb
FOUND ONE! D
FOUND ONE! G
NOTHING WORKED
FOUND ONE! F
FOUND ONE! F
NOTHING WORKED
FOUND ONE! Eb
NOTHING WORKED
FOUND ONE! A
FOUND ONE! G
NOTHING WORKED
FOUND ONE! D
FOUND ONE! C
FOUND ONE! D
NOTHING WORKED
FOUND ONE! A
NOTHING WORKED
FOUND ONE! G
FOUND ONE! Eb
NOTHING WORKED
FOUND ONE! C
NOTHING WORKED
NOTHING WORKED
FOUND ONE! G
FOUND ONE! C
NOTHING WORKED
FOUND ONE! C
NOTHING WORKED
FOUND ONE! G
FOUND ONE! Eb
FOUND ONE! G
FOUND ONE! F
FOUND ONE! G
FOUND ONE! D
FOUND ONE! G
NOTHING WORKED
FOUND ONE! F
FOUND ONE! C
FOUND ONE! A
FOUND ONE! C
NOTHING WORKED
FOUND ONE! C
FOUND ONE! C
FOUND ONE! Eb
FOUND ONE! C
FOUND ONE! E
FOUND ONE! Eb
FOUND ONE! Eb
FOUND ONE! E
FOUND ONE! F
FOUND ONE! A
FOUND ONE! D
FOUND ONE! G
FOUND ONE! Eb
FOUND ONE! Eb
FOUND ONE! F
FOUND ONE! D
FOUND ONE! Ab
FOUND ONE! A
FOUND ONE! D
NOTHING WORKED
FOUND 

In [58]:
def major_or_minor(chorale):
    final_chord = sorted(list(set([i%12 for i in chorale[-1]])))
    print(final_chord)
    if 12 - max(final_chord) <= min(final_chord):
        final_chord = sorted([(i + (12 - max(final_chord)))%12 for i in final_chord])
    print(final_chord)
    # need distances between notes
    if final_chord[1] - final_chord[0] == 4:
        return 'major'
    else: 
        return 'minor'

majors = 0
minors = 0
for i in range(len(chorale_data['train'])):
    if major_or_minor(chorale_data['train'][i]) == 'major':
        majors+=1
    else: 
        minors +=1
print("MAJORS:", majors, "MINORS", minors)

[2, 5, 10]
[0, 4, 7]
[1, 4, 9]
[1, 4, 9]
[2, 7, 11]
[0, 3, 8]
[0, 4, 7]
[0, 4, 7]
[4, 8, 11]
[0, 5, 9]
[3, 6, 11]
[0, 4, 7]
[3, 7, 10]
[0, 5, 9]
[2, 6, 9]
[2, 6, 9]
[4, 8, 11]
[0, 5, 9]
[2, 7, 11]
[0, 3, 8]
[2, 6, 9]
[2, 6, 9]
[2, 5, 10]
[0, 4, 7]
[4, 8, 11]
[0, 5, 9]
[2, 7, 11]
[0, 3, 8]
[2, 6, 9]
[2, 6, 9]
[0, 5, 9]
[0, 5, 9]
[0, 5, 9]
[0, 5, 9]
[3, 7, 10]
[0, 5, 9]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[1, 4, 9]
[1, 4, 9]
[2, 7, 11]
[0, 3, 8]
[3, 7, 10]
[0, 5, 9]
[3, 6, 11]
[0, 4, 7]
[2, 6, 9]
[2, 6, 9]
[2, 6, 11]
[0, 3, 7]
[1, 4, 9]
[1, 4, 9]
[1, 4, 9]
[1, 4, 9]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[2, 5, 10]
[0, 4, 7]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[1, 4, 9]
[1, 4, 9]
[2.0, 7.0, 11.0]
[0.0, 3.0, 8.0]
[4, 8, 11]
[0, 5, 9]
[3.0, 7.0, 10.0]
[0.0, 5.0, 9.0]
[1, 4, 9]
[1, 4, 9]
[2, 6, 9]
[2, 6, 9]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[0, 5, 9]
[0, 5, 9]
[2.0, 7.0, 11.0]
[0.0, 3.0, 8.0]
[2.0, 6.0, 9.0]
[2.0, 6.0, 9.0]
[4

In [9]:
single_training_chorale = chorale_data['train'][0]
print(len(single_training_chorale))

96


In [68]:
for notes in chorale_data['train'][1]:
    print(notes)

[69, 64, 61, 57]
[69, 64, 61, 57]
[71, 64, 59, 56]
[71, 64, 57, 54]
[71, 64, 56, 52]
[71, 64, 56, 50]
[69, 64, 57, 49]
[69, 64, 57, 49]
[66, 62, 57, 50]
[66, 61, 57, 50]
[64, 59, 56, 50]
[64, 59, 56, 50]
[66, 61, 57, 49]
[68, 62, 59, 47]
[69, 64, 52, 49]
[69, 64, 57, 45]
[71, 64, 57, 52]
[71, 64, 56, 52]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[73, 69, 64, 57]
[74, 69, 64, 57]
[76, 71, 64, 56]
[76, 73, 64, 56]
[78, 74, 57, 54]
[78, 73, 57, 54]
[76, 71, 59, 56]
[76, 71, 59, 56]
[73, 69, 61, 57]
[73, 69, 62, 57]
[69, 69, 64, 49]
[69, 69, 64, 49]
[66, 62, 57, 50]
[68, 62, 57, 52]
[69, 61, 57, 54]
[69, 61, 57, 57]
[71, 66, 57, 50]
[71, 64, 56, 52]
[69, 61, 52, 45]
[69, 61, 52, 45]
[69, 61, 52, 45]
[69, 61, 52, 45]
[73, 69, 64, 57]
[73, 69, 64, 56]
[73, 71, 68, 54]
[73, 71, 68, 53]
[73, 69, 66, 54]
[73, 68, 66, 52]
[71, 66, 54, 50]
[71, 66, 56, 50]
[73, 66, 57, 49]
[73, 66, 57, 49]
[73, 65, 56, 49]
[73, 65, 56, 49]
[69, 61, 54, 5

In [97]:
major_to_minor_key = {
    'C':'a',
    'G':'e',
    'D':'b',
    'A':'f#',
    'E':'c#',
    'B/Cb':'ab/g#',
    'Gb/F#':'eb/d#',
    'Db/C#':'bb/a#',
    'F':'d',
    'Bb':'g',
    'Eb':'c',
    'Ab':'f',
    'Gb':'eb',
}

tonic_chords_for_keys = {
    'C':[0,4,7], # ceg
    'a':[9,0,4], # ace
    'G':[7,11,2], # gbd
    'e':[4,7,11],
    'D':[2,6,9],
    'b':[11,2,6],
    'A':[9,1,4],
    'f#':[6,10,1],
    'E':[4,8,11],
    'c#':[1,4,8],
    'B/Cb':[11,3,6],
    'ab/g#':[8,11,3],
    'Gb/F#':[6,10,1],
    'eb/d#':[3,6,10],
    'Db/C#':[0,4,7],
    'bb/a#':[10,1,5],
    'F':[5,9,0],
    'd':[2,5,9], # dfa
    'Bb':[10,2,5],
    'g':[7,10,2],
    'Eb':[3,7,10],
    'c':[0,3,7],
    'Ab':[8,0,3],
    'f':[5,8,0],
}

notes_in_keys = {
    #        C D E F G A B
    "C":    [0,2,4,5,7,9,11], # 7 notes in the key of c!
    "G":    [0,2,4,6,7,9,11], #  F#
    "D":    [1,2,4,6,7,9,11], # +C#
    "A":    [1,2,4,6,8,9,11], # +G#
    "E":    [1,3,4,6,8,9,11], # +D#
    "B/Cb": [1,3,4,6,8,10,11], # +A#
    "Gb/F#":[1,3,5,6,8,10,11], # +E#
    "Db/C#":[0,1,3,5,6,8,10], # +B#
    "Ab":   [0,1,3,5,7,8,10], # +Db
    "Eb":   [0,2,3,5,7,8,10], # +Ab
    "Bb":   [0,2,3,5,7,9,10], # +Eb 
    "F":    [0,2,4,5,7,9,10]  # +Bb
}

raised_7th_for_minor_keys = {
    #       [0,2,4,5,7,9,11]
    #        C D E F G A B
    "C":    8, # a minor, raised 7th is G-->G#
    "G":    3, #  e minor, raised 7th is D-->D#
    "D":    10, # Bm, raised 7th is A-->A#
    "A":    5, # f# minor, raised 7th is E-->F
    "E":    0, # c# minor, raised 7th is B-->C
    "B/Cb": 7, # g# minor, raised 7th is F#-->G
    "Gb/F#":2, # d# minor, raised 7th is C#-->D
    "Db/C#":9, # bb minor, raised 7th is Ab --> A
    "Ab":   4, # f minor, raised 7th is Eb --> E
    "Eb":   11, # c minor, raised 7th is Bb --> B
    "Bb":   6, # g minor, raised 7th is F-->F#
    "F":    1,  # d minor, raised 7th is C-->C#
}

def estimate_key(chorale):
#chorale = chorale data['train'][1]
    mode = "maj"
    note_counts = [0,0,0,0,0,0,0,0,0,0,0,0]
    for chord in chorale: 
        for note in chord: 
            note_counts[int(note)%12] += 1
    chosen_key = "C"
    max_key_count = 0
    max_key_count_raised = 0
    for key in notes_in_keys.keys():
        key_count = 0
        for note in notes_in_keys[key]:
            key_count += note_counts[note]
        key_count_with_raised_7th = key_count + note_counts[raised_7th_for_minor_keys[key]]
        if key_count > max_key_count:
            #print("NEW MAX", key, key_count)
            chosen_key = key
            max_key_count = key_count
            max_key_count_raised = key_count_with_raised_7th
    #print("Key:", chosen_key, max_key_count_raised/sum(note_counts) - max_key_count/sum(note_counts))
    if max_key_count_raised/sum(note_counts) - max_key_count/sum(note_counts) > 0.0137:
        key = major_to_minor_key[chosen_key]
        mode = "min"
    return chosen_key, mode

mins = 0
print("NUMBER OF CHORALES:", len(chorale_data['train']))
for chorale in chorale_data['train']:
    key, mode = estimate_key(chorale)
    if mode == 'min':
        mins += 1
print("TOTAL MINS:", mins) # should be 121


NUMBER OF CHORALES: 229
TOTAL MINS: 121


In [32]:
sorted_notecounts = key_estimate(chorale_data['train'][0])
# 3 = D#/Eb, 10=A#Bb #FCGDAEB #BEADGCF FLATS
# SO, BE, TWO FLATS!

{5: 80, 10: 77, 3: 50, 2: 49, 7: 49, 0: 48, 9: 24, 11: 4, 8: 3} 
 [5, 10, 3, 2, 7, 0, 9]


NameError: name 'x' is not defined